In [ ]:
import time
import numpy as np

import _init_paths
from libs.scheduler import PfScheduler
from libs.models.naive_solver import NaiveURLLCSolver
from libs.models.greedy_solver import GreedyURLLCSolver
from libs.models.nb_solver import CccpURLLCSolver
from utils.generator import generate
from utils.generator import urllc_generate
from utils.metrics import get_embb_utility

import matplotlib.pyplot as plt
import math

import matlab
import matlab.engine

In [ ]:
utility_result = []
embb_user_result = []
urllc_num_slot = []
utility_result_mid = []
engine = matlab.engine.start_matlab()

In [ ]:
for i in range(6):
    urllc_per_slot = i

    rb_size = 15
    rb_num = 254
    embb_num = 3
    embb_slot_len = 7

    urllc_slot_len = 1
    max_sim_time_slot_len = embb_slot_len
    id_total = 0
    embb_rb_req = [100, 50, 120]
    embb_rb_size = [6000, 12000, 5000]
    urllc_rb_req = []
    urllc_rb_size = []
    urllc_slot_start = []
    embb_utility = 0

    urllc_num = (embb_slot_len - 1) * urllc_per_slot
    for i in range(embb_slot_len-1):
        for k in range(urllc_per_slot):
            urllc_rb_req.append(15)
            urllc_rb_size.append(30)
            urllc_slot_start.append(i+1)
    print(urllc_slot_start)
    print(urllc_rb_req)
    print(urllc_rb_size)
    urllc_num_slot.append(urllc_per_slot)

    #urllc scheduler setting
    #URLLCSolver = NaiveURLLCSolver
    #URLLCSolver = GreedyURLLCSolver
    URLLCSolver = CccpURLLCSolver
    global_time = 0
    global_timeout_urllc_users = []
    
    # generate eMBB
    embb_users, urllc_users, RB_map, id_total = generate(rb_size,
        rb_num,
        embb_num, 
        embb_slot_len, 
        urllc_num,
        urllc_slot_len,
        embb_rb_req,
        embb_rb_size,
        urllc_rb_req,
        urllc_rb_size,
        urllc_slot_start,
        latency=1,
        error_rate=1e-5,
        mcs_error=1e-5,
    )
    delay_users = []
    timer = []
    
    for i in range(30):
        pf_scheduler = PfScheduler(RB_map, embb_users)
        pf_scheduler.allocate_resource()



        # urllc_time_slot get urllc_active_come_list, setting urllc come from global_time=1 and later
        urllc_users.sort(key=lambda x:x.slot_start)
        urllc_come_time = np.array([u.slot_start for u in urllc_users])
        #print(urllc_come_time)
        #print(global_time)
        #print(max_sim_time_slot_len)
        while(global_time < max_sim_time_slot_len):
            global_time += 1
            #print(global_time)
            indexes = np.where(urllc_come_time==global_time)[0]
            if len(indexes) == 0:
                #print(indexes)
                continue
            urllc_users_list = delay_users + urllc_users.copy()[indexes[0]:indexes[-1]+1]


            urllc_scheduler = URLLCSolver(RB_map, embb_users, urllc_users_list, engine)

            start = time.time()
            ass_users, delay_users, timeout_users = urllc_scheduler.allocate_resource()
            timer.append(time.time() - start)

            global_timeout_urllc_users.append(timeout_users)

            #print(RB_map.__dict__.items())
            # urllc leave
            urllc_scheduler.leave(ass_users)

    # get_embb_utility miss_list and time cost for all the urllc scheduler
        embb_utility = get_embb_utility(embb_users)
        total_time_cost = sum(timer)

        # visualize (TODO)
        for user in embb_users:
            print(user .__dict__.items())
        
        print("\n")
        print(embb_utility)
        #print(RB_map.__dict__.items())

        # next embb_time_slot (not show here)

        #update URLLC
        for i in range(len(urllc_slot_start)):
            urllc_slot_start[i] += embb_slot_len


        urllc_users, id_total = urllc_generate(
            urllc_num,
            urllc_slot_len,
            urllc_rb_req,
            urllc_rb_size,
            urllc_slot_start,
            id_total,
            latency=1,
            error_rate=1e-5,
            mcs_error=1e-5,
        )

        max_sim_time_slot_len+=embb_slot_len
        avg_utiliy_mid = 0
        for user in embb_users:
            weighti = user.DRC
            if user.sche_times:
                avg_utiliy_mid += weighti * math.log(user.rate_avg)
        utility_result_mid.append(avg_utiliy_mid)
    avg_utiliy = 0
    for user in embb_users:
        weighti = user.DRC
        if user.sche_times:
            avg_utiliy += weighti * math.log(user.rate_avg)
    utility_result.append(avg_utiliy)
    embb_user_result.append(embb_users)

In [ ]:
utility_result

In [ ]:
utility_result_mid

In [ ]:
avg_utiliy = 0
for user in embb_users:
    print(user .__dict__.items())
    weighti = user.DRC
    if user.sche_times:
        avg_utiliy += weighti * math.log(user.rate_avg)
print(embb_utility)
print(utility_result)
print(avg_utiliy)

## Test 

In [ ]:
utility_result = []
embb_user_result = []
urllc_num_slot = []
utility_result_mid = []

In [ ]:
for i in range(6):
    urllc_per_slot = i

    rb_size = 15
    rb_num = 274
    embb_num = 5
    embb_slot_len = 7

    urllc_slot_len = 1
    max_sim_time_slot_len = embb_slot_len
    id_total = 0
    embb_rb_req = [60, 80, 40,50,70]
    embb_rb_size = [7000, 5200, 10000, 8400, 6000]
    urllc_rb_req = []
    urllc_rb_size = []
    urllc_slot_start = []
    embb_utility = 0

    urllc_num = (embb_slot_len - 1) * urllc_per_slot
    for i in range(embb_slot_len-1):
        for k in range(urllc_per_slot):
            urllc_rb_req.append(15)
            urllc_rb_size.append(30)
            urllc_slot_start.append(i+1)
    print(urllc_slot_start)
    print(urllc_rb_req)
    print(urllc_rb_size)
    urllc_num_slot.append(urllc_per_slot)

    #urllc scheduler setting
    #URLLCSolver = NaiveURLLCSolver
    #URLLCSolver = GreedyURLLCSolver
    URLLCSolver = CccpURLLCSolver
    global_time = 0
    global_timeout_urllc_users = []
    
    # generate eMBB
    embb_users, urllc_users, RB_map, id_total = generate(rb_size,
        rb_num,
        embb_num, 
        embb_slot_len, 
        urllc_num,
        urllc_slot_len,
        embb_rb_req,
        embb_rb_size,
        urllc_rb_req,
        urllc_rb_size,
        urllc_slot_start,
        latency=1,
        error_rate=1e-5,
        mcs_error=1e-5,
    )
    delay_users = []
    timer = []
    
    for i in range(30):
        pf_scheduler = PfScheduler(RB_map, embb_users)
        pf_scheduler.allocate_resource()



        # urllc_time_slot get urllc_active_come_list, setting urllc come from global_time=1 and later
        urllc_users.sort(key=lambda x:x.slot_start)
        urllc_come_time = np.array([u.slot_start for u in urllc_users])
        #print(urllc_come_time)
        #print(global_time)
        #print(max_sim_time_slot_len)
        while(global_time < max_sim_time_slot_len):
            global_time += 1
            #print(global_time)
            indexes = np.where(urllc_come_time==global_time)[0]
            if len(indexes) == 0:
                #print(indexes)
                continue
            urllc_users_list = delay_users + urllc_users.copy()[indexes[0]:indexes[-1]+1]


            urllc_scheduler = URLLCSolver(RB_map, embb_users, urllc_users_list, engine)

            start = time.time()
            ass_users, delay_users, timeout_users = urllc_scheduler.allocate_resource()
            timer.append(time.time() - start)

            global_timeout_urllc_users.append(timeout_users)

            #print(RB_map.__dict__.items())
            # urllc leave
            urllc_scheduler.leave(ass_users)

    # get_embb_utility miss_list and time cost for all the urllc scheduler
        embb_utility = get_embb_utility(embb_users)
        total_time_cost = sum(timer)

        # visualize (TODO)
        for user in embb_users:
            print(user .__dict__.items())
        
        print("\n")
        print(embb_utility)
        #print(RB_map.__dict__.items())

        # next embb_time_slot (not show here)

        #update URLLC
        for i in range(len(urllc_slot_start)):
            urllc_slot_start[i] += embb_slot_len


        urllc_users, id_total = urllc_generate(
            urllc_num,
            urllc_slot_len,
            urllc_rb_req,
            urllc_rb_size,
            urllc_slot_start,
            id_total,
            latency=1,
            error_rate=1e-5,
            mcs_error=1e-5,
        )

        max_sim_time_slot_len+=embb_slot_len
        avg_utiliy_mid = 0
        for user in embb_users:
            weighti = user.DRC
            if user.sche_times:
                avg_utiliy_mid += weighti * math.log(user.rate_avg)
        utility_result_mid.append(avg_utiliy_mid)
    avg_utiliy = 0
    for user in embb_users:
        weighti = user.DRC
        if user.sche_times:
            avg_utiliy += weighti * math.log(user.rate_avg)
    utility_result.append(avg_utiliy)
    embb_user_result.append(embb_users)